In [26]:
import pandas as pd

# Upload data
from google.colab import files
uploaded = files.upload()

# Load and show data
df = pd.read_csv(next(iter(uploaded)))
df.head()

Saving employee_data.csv to employee_data (4).csv


,Education,Experience,Role,Department,Location,Salary
0,Masters,20,Data Scientist,Finance,New York,119127
1,PhD,0,Software Engineer,IT,San Francisco,112569
2,High School,2,HR,Finance,Austin,47784
3,Masters,17,Manager,Finance,San Francisco,116583
4,Masters,9,Data Scientist,Marketing,San Francisco,104946


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Features and target
X = df.drop('Salary', axis=1)
y = df['Salary']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Test accuracy
preds = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, preds))

MSE: 9215659.312638685


In [29]:
import pickle

with open('salary_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [30]:
from google.colab import files
files.download('salary_model.pkl')
files.download('encoders.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
%%writefile app.py
import streamlit as st
import pickle

import pandas as pd
import numpy as np
import streamlit as st
import pickle

# Load dataset
df = pd.read_csv("employee_data.csv")

# One-hot encode categorical features
X = pd.get_dummies(df.drop("Salary", axis=1))
y = df["Salary"]

# Save column names for use in app.py
import pickle
with open("columns.pkl", "wb") as f:
    pickle.dump(X.columns.tolist(), f)

# Train model
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X, y)

# Save model
with open("salary_model.pkl", "wb") as f:
    pickle.dump(model, f)
    import streamlit as st
import pandas as pd
import pickle

# Load model and columns
model = pickle.load(open("salary_model.pkl", "rb"))
columns = pickle.load(open("columns.pkl", "rb"))

# User input
education = st.selectbox("Education", ["High School", "Bachelors", "Masters", "PhD"])
experience = st.slider("Years of Experience", 0, 40)
role = st.selectbox("Role", ["Software Engineer", "Data Scientist", "Manager", "HR"])
department = st.selectbox("Department", ["IT", "Finance", "Marketing", "HR"])
location = st.selectbox("Location", ["New York", "San Francisco", "Austin", "Chicago"])

# Prepare input
input_df = pd.DataFrame({
    'Education': [education],
    'Experience': [experience],
    'Role': [role],
    'Department': [department],
    'Location': [location]
})

# One-hot encode to match training
input_encoded = pd.get_dummies(input_df)
input_encoded = input_encoded.reindex(columns=columns, fill_value=0)

# Predict
if st.button("Predict Salary"):
    prediction = model.predict(input_encoded)[0]
    st.success(f"Estimated Salary: ₹{prediction:,.2f}")

Overwriting app.py


In [32]:
X = pd.get_dummies(df.drop("Salary", axis=1))
y = df["Salary"]

# Save column names used during training
with open("columns.pkl", "wb") as f:
    pickle.dump(X.columns.tolist(), f)

In [33]:
!pip install streamlit pyngrok --quiet

In [34]:
from pyngrok import ngrok

# Set your ngrok authtoken here
ngrok.set_auth_token("308pTXWLeRajppH5YO4b906ItU2_2AR1tgb1meW6PfFhkmcgM")

In [35]:
from pyngrok import ngrok
import os
import time

# Paste your real token here
ngrok.set_auth_token("308pTXWLeRajppH5YO4b906ItU2_2AR1tgb1meW6PfFhkmcgM")

# Kill previous tunnels
ngrok.kill()
time.sleep(2) # Add a small delay

# Start Streamlit
os.system("streamlit run app.py &")
time.sleep(5)

# Tunnel the app
public_url = ngrok.connect(8501)
print("✅ Your app is live at:", public_url)

✅ Your app is live at: NgrokTunnel: "https://71148690549e.ngrok-free.app" -> "http://localhost:8501"
